In [1]:
import os
from env.audit import Audits

appsecret = os.getenv("admin_appsecret")
appid = os.getenv("admin_appid")
tenantid = os.getenv("admin_tenantid")

audit = Audits()
audit.set_ServicePrincipal(
    tenant_id=tenantid,
    client_id=appid,
    client_secret= appsecret
)


In [2]:

# audit.set_ApplicationModules("Apps")
audit.set_ApplicationModules("Activity,Apps,Capacity,Catalog,Domains,FabricItems,Gateway,Graph,Refreshables,RefreshHistory,Roles,Tenant,Workspaces")
# audit.set_ApplicationModules("Activity,Apps")
# audit.set_ApplicationModules("Roles,Tenant,Workspaces")

# a list of the supported logging levels
import logging
Logging_levels = {
            "DEBUG": logging.DEBUG,
            "INFO": logging.INFO,
            "WARNING": logging.WARNING,
            "ERROR": logging.ERROR,
            "CRITICAL": logging.CRITICAL
}

audit.set_LakehouseName("FabricLake")
audit.set_PathInLakehouse("stage")
audit.set_WorkspaceName("FabricMonitor")
audit.set_logging_config("INFO", "brandon.log")
audit.set_all_workspaces(True)

audit.set_Activity_cron("* * * * * 30")
audit.set_Catalog_cron("* * * * * 30")
audit.set_Capacity_cron("* * * * * 30")
audit.set_Apps_cron("* * * * * 30")
audit.set_Domains_cron("* * * * * 30") # Not yet implemented
audit.set_Graph_cron("* * * * * 30")
audit.set_Tenant_cron("* * * * * 30")
audit.set_RefreshHistory_cron("* * * * * 30")
audit.set_Refreshables_cron("* * * * * 30")
audit.set_Gateway_cron("* * * * * 30")

# print(audit.ServicePrincipal)
await audit.run()



2024-09-16 11:06:13,694 - azure.identity._internal.get_token_mixin - WARNING - ClientSecretCredential.get_token failed: Authentication failed: AADSTS7000215: Invalid client secret provided. Ensure the secret being sent in the request is the client secret value, not the client secret ID, for a secret added to app 'e15fadf9-bc05-4ade-af9f-d79a918bbacd'. Trace ID: 3b7a12a8-c5c6-48c7-89a7-ef6d82bc3f00 Correlation ID: d821e10c-559d-4fd8-aab3-1b8fb31d3cb2 Timestamp: 2024-09-16 15:06:13Z
2024-09-16 11:06:32,863 - azure.identity._internal.get_token_mixin - WARNING - ClientSecretCredential.get_token failed: Authentication failed: AADSTS7000215: Invalid client secret provided. Ensure the secret being sent in the request is the client secret value, not the client secret ID, for a secret added to app 'e15fadf9-bc05-4ade-af9f-d79a918bbacd'. Trace ID: ab675d1f-98ce-4bdf-8bde-88267c563b00 Correlation ID: 03c197c2-82cd-45a0-bc0a-c872c85d90f1 Timestamp: 2024-09-16 15:06:32Z
2024-09-16 11:06:56,255 - az

## UPSERT Example

In [ ]:
from delta.tables import DeltaTable

# Assuming you have a Delta table named "my_delta_table"
delta_table_path = "path/to/delta-table"  # Replace with your actual path

# Create or load your DataFrame (myDataFrame) with the data to be upserted
# For demonstration purposes, let's assume you have columns: "Id", "Type", "Value"

# Initialize the DeltaTable
# delta_table = DeltaTable.forPath(spark, delta_table_path)
delta_table = spark.read.format("delta").load(delta_table_path)
myDataFrame = spark.read.format("delta").load(myDataFrame_path)

# Merge the updates into the Delta table
delta_table.alias("events") \
    .merge(myDataFrame.alias("updates"), "events.Id = updates.Id and events.Type = updates.Type") \
    .whenMatchedUpdateAll() \
    .whenNotMatchedInsertAll() \
    .execute()

# Commit the changes
delta_table.toDF().write.format("delta").mode("overwrite").save(delta_table_path)


In [ ]:
from pyspark.sql import SparkSession 
from pyspark.sql.types import StructType, StructField, StringType, TimestampType
from pyspark.sql.functions import lit
from pyspark import SparkConf


In [ ]:
sparkConf = SparkConf()
sparkConf.set("fs.azure.account.auth.type.onelake.dfs.core.windows.net", "OAuth")
sparkConf.set("fs.azure.account.oauth.provider.type.onelake.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
sparkConf.set("fs.azure.account.oauth2.client.id.onelake.dfs.core.windows.net", appid)
sparkConf.set("fs.azure.account.oauth2.client.secret.onelake.dfs.core.windows.net", appsecret)
sparkConf.set("fs.azure.account.oauth2.client.endpoint.onelake.dfs.core.windows.net", "https://login.microsoftonline.com/" + tenantid)
sparkConf.set("spark.jars.packages", "org.apache.hadoop:hadoop-azure:3.3.1")
spark = SparkSession.builder.config(conf=sparkConf).appName("SparkTest").getOrCreate()


schema = StructType([
    StructField("name", StringType(), True),
    StructField("age", StringType(), True),
    StructField("timestamp", TimestampType(), True)
])

import datetime

now = datetime.datetime.now()

data = [("John", "30", now), ("Jane", "25", now)]
df = spark.createDataFrame(data, schema)
df = df.withColumn("source", lit("test"))

df.write.format("parquet").mode("append").save("abfss://onelake.dfs.core.windows.net/FabricLake.Lakehouse/Files/data.parquet")
# df.write.format("parquet").mode("append").save("FabricLake.Lakehouse/Files/data.parquet")
df.show()



In [ ]:
# df.write.format("delta").mode("append").saveAsTable("data")
df.write.format("parquet").save("data.parquet")

In [ ]:
# Instantiate a DataLakeServiceClient using a connection string
   from azure.storage.filedatalake import DataLakeServiceClient
   datalake_service_client = DataLakeServiceClient.from_connection_string(self.connection_string)

   # Instantiate a FileSystemClient
   file_system_client = datalake_service_client.get_file_system_client("mynewfilesystem")

In [ ]:
from azure.identity import DefaultAzureCredential
from azure.storage.filedatalake import DataLakeServiceClient, FileSystemClient
from azure.identity import ClientSecretCredential
import datetime
import json

storage_account_name = "onelake"
workspace_name = "FabricMonitor"

credential = ClientSecretCredential(tenant_id=tenantid,
                            client_id=appid,
                            client_secret=appsecret)

file_system_client = FileSystemClient(
    account_url="https://onelake.dfs.fabric.microsoft.com",
    file_system_name=workspace_name,
    credential=credential)

dc = file_system_client.get_directory_client("FabricLake.Lakehouse/Files")
file_client = dc.get_file_client("data.parquet")
with open("./data.parquet", "rb") as data:
    file_client.upload_data(data, overwrite=True)

    


df.write.format("delta").mode("append").save("abfss://onelake.dfs.core.windows.net/FabricLake.Lakehouse/Files/data.parquet")


In [ ]:
import pandas as pd
# Load data into pandas DataFrame from "/lakehouse/default/" + "Files/stage/state.json"
state = pd.read_json("/lakehouse/default/" + "Files/stage/state.yaml",typ="series")

lastRun = state['activity']['lastRun']
lastRun


In [ ]:
with open("/lakehouse/default/" + "Files/stage/state.yaml", "r") as file:
    for line in file:
        print(line)

In [ ]:
s = spark.read.option("multiline", "true").json("Files/stage/state.json")
df = spark.read.option("multiLines","json").json("Files/stage/catalog/scans/2024/07/22/*.json")

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from functools import reduce

data = df.withColumn("datasets", explode(df["workspaces"]["datasets"]))

df = df.withColumn("start_Time", to_utc_timestamp(df['startTime'], "UTC"))
df = df.withColumn("cst_timestamp", from_utc_timestamp(df.start_Time, "America/Chicago"))
df = df.withColumn("eu_timestamp", from_utc_timestamp(df.start_Time, "Europe/Berlin"))
df = df.withColumn("ist_timestamp", from_utc_timestamp(df.start_Time, "Asia/Kolkata"))


In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from functools import reduce
from  pyspark.sql.functions import input_file_name

dc = df.withColumn("requestId", df.requestId)\
    .withColumn("refreshAttempts", explode(df.refreshAttempts))\
    .select("requestId","refreshAttempts")

display(dc)

In [ ]:
de = dc.withColumn("attemptId", dc.refreshAttempts.attemptId)\
    .withColumn("startTime", dc.refreshAttempts.startTime)\
    .withColumn("endTime", dc.refreshAttempts.endTime)\
    .withColumn("type", dc.refreshAttempts.type)\
    .select("requestId","attemptId","startTime","endTime","type")

display(de)

In [ ]:
df.select('dataset_id', 'dataset_name', 'endTime', 'id', 'refreshType', 'requestId', 'serviceExceptionJson', 'startTime', 'status').dropDuplicates(["dataset_id","requestId"]).write.format("delta").mode("overwrite").option("overwriteSchema","true").save("Tables/refreshhistory")
de.write.format("delta").mode("overwrite").option("overwriteSchema","true").save("Tables/refreshhistory_attempts")

In [ ]:
%%sql

SELECT dataset_id, COUNT(DISTINCT requestId) as requestId_count
FROM refreshhistory
GROUP BY dataset_id
HAVING COUNT(DISTINCT requestId) > 1


### Get the latest record by date

In [ ]:
ds = "Tables/workspaces_workspaces"
df = spark.read.format("delta").load(ds)
dd = df.withColumn("scandate", concat_ws("-",df.ts_year, df.ts_month, df.ts_day).cast("date"))

# Define window specification
window_spec = Window.partitionBy("id").orderBy(col("scandate").desc())

# Add row number to each partition
df_with_row_num = dd.withColumn("row_number", row_number().over(window_spec))

# Filter to get the latest records for each id
latest_records_df = df_with_row_num.filter(col("row_number") == 1).drop("row_number")

# Show the latest records DataFrame
latest_records_df.write.format("delta").mode("overwrite").option("overwriteSchema","true").save(ds)

In [ ]:
df = spark.read.option("multiline","true").json("Files/stage/catalog/scans/2024/*/*/*.json")
dd = df.withColumn("workspaces", explode(df.workspaces)).drop("datasourceInstances")

workspaces = dd.withColumn("capacityId", dd["workspaces"]["capacityId"]) \
    .withColumn("dataRetrievalState", dd["workspaces"]["dataRetrievalState"]) \
    .withColumn("defaultDatasetStorageFormat", dd["workspaces"]["defaultDatasetStorageFormat"]) \
    .withColumn("description", dd["workspaces"]["description"]) \
    .withColumn("domainId", dd["workspaces"]["domainId"]) \
    .withColumn("id", dd["workspaces"]["id"]) \
    .withColumn("isOnDedicatedCapacity", dd["workspaces"]["isOnDedicatedCapacity"]) \
    .withColumn("name", dd["workspaces"]["name"]) \
    .withColumn("state", dd["workspaces"]["state"]) \
    .withColumn("type", dd["workspaces"]["type"]) 


ds = dd.withColumn("id", dd.workspaces.id)\
    .withColumn("datasets", explode(dd.workspaces.datasets)).drop("workspaces")

ds = ds.withColumn("dataset_id", ds["datasets"]["id"])

ds.select("id","dataset_id").dropDuplicates().write.format("delta").mode("overwrite").save("Tables/catalog_workspace_datasets")

ee = spark.read.format("delta").load("Tables/datasetrefreshable_datasetrefreshable")
display(ee)

workspaces = dd.withColumn("id", dd["workspaces"]["id"]) \
            .withColumn("defaultDatasetStorageFormat",dd["workspaces"]["defaultDatasetStorageFormat"])\
            .withColumn("description",dd["workspaces"]["description"])\
            .withColumn("domainId",dd["workspaces"]["domainId"])\
            .withColumn("isOnDedicatedCapacity",df["workspaces"]["isOnDedicatedCapacity"])\
            .withColumn("datasets",explode(df["workspaces"]["datasets"]))

dd = df.withColumn("workspaces", explode(df.workspaces)).drop("datasourceInstances")
display(dd)

display(dd.select(dd["workspaces"]["id"]))

df = spark.read.format("delta").load("Tables/catalog_datasets")
display(df.limit(10))

datasets = df.withColumn("scandate", concat_ws("-",df.ts_year, df.ts_month, df.ts_day).cast("date"))
display(datasets.limit(10))

In [ ]:
ds = "Tables/workspaces_workspaces"
df = spark.read.format("delta").load(ds)
dd = df.withColumn("scandate", concat_ws("-",df.ts_year, df.ts_month, df.ts_day).cast("date"))

# Define window specification
window_spec = Window.partitionBy("id").orderBy(col("scandate").desc())

# Add row number to each partition
df_with_row_num = dd.withColumn("row_number", row_number().over(window_spec))

# Filter to get the latest records for each id
latest_records_df = df_with_row_num.filter(col("row_number") == 1).drop("row_number")

# Show the latest records DataFrame
latest_records_df.write.format("delta").mode("overwrite").option("overwriteSchema","true").save(ds)

latest_records_df.write.format("delta").save("Tables/catalog_datasets_unique")

df = spark.read.format("delta").load("Tables/catalog_datasets_tables")
df_tables = df.withColumn("scandate", concat_ws("-",df.ts_year, df.ts_month, df.ts_day).cast("date"))

df = spark.read.format("delta").load("Tables/catalog_datasets_columns")
df_columns = df.withColumn("scandate", concat_ws("-",df.ts_year, df.ts_month, df.ts_day).cast("date"))

df = spark.read.format("delta").load("Tables/catalog_datasets_measures")
df_measures = df.withColumn("scandate", concat_ws("-",df.ts_year, df.ts_month, df.ts_day).cast("date"))



In [ ]:
measures = df_measures.withColumn("UID", concat_ws("-", col("datasets_id"), col("datasets_tables_name"), col("scandate")))
tables = df_tables.withColumn("UID", concat_ws("-", col("datasets_id"), col("datasets_tables_name"), col("scandate")))
columns = df_columns.withColumn("UID", concat_ws("-", col("datasets_id"), col("datasets_tables_name"), col("scandate")))

measures.write.format("delta").save("Tables/catalog_datasets_measures_history")
tables.dropDuplicates("UID").write.format("delta").save("Tables/catalog_datasets_tables_history")
columns.write.format("delta").save("Tables/catalog_datasets_columns_history")

In [ ]:
dd = spark.read.format("delta").load("Tables/catalog_datasets_columns_history")
display(dd.limit(10))

In [ ]:
# tables = spark.read.format("delta").load("Tables/catalog_datasets_tables_history")
cols = ["datasets_id","datasets_tables_name","scandate"]
display(tables.orderBy(*cols).select(*cols))

In [ ]:
dd.dropDuplicates(["UID"]).write.format("delta").mode("overwrite").option("overwriteSchema", "True").save("Tables/catalog_datasets_tables_history")

In [ ]:
%%sql

select T.UID, datasets_tables_name, datasets_id, scandate, description, name FROM catalog_tables_history as T
INNER JOIN (select UID, COUNT(UID) FROM catalog_tables_history group by UID having count(UID)>1) as G
ON T.UID=G.UID

In [ ]:
cols = ["datasets_id", "datasets_tables_name"]

window_spec = Window.partitionBy(*cols).orderBy(col("scandate").desc())

# Add row number to each partition
df_with_row_num = df_tables.withColumn("row_number", row_number().over(window_spec))

# Filter to get the latest records for each id
latest_records_df = df_with_row_num.filter(col("row_number") == 1).drop("row_number")

# Show the latest records DataFrame
latest_records_df.select(col("datasets_id"), col("datasets_tables_name"), col("scandate")).show()